<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/langchain_index_zapier_agent_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**! This Colab has a video tutorial. Find it here https://youtu.be/1towAoXOWLg**

# Setting up

Install dependencies

In [ ]:
!pip install langchain==0.0.157
!pip install pypdf
!pip install pinecone-client
!pip install openai
!pip install tiktoken
!pip install chromadb

Set up OpenAI API key

In [ ]:
import os 
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

Set up Pinecone API keys

In [ ]:
import pinecone 

# initialize pinecone
pinecone.init(
    api_key="YOUR_PINECONE_API_KEY",  # find at app.pinecone.io
    environment="YOUR_ENVIRONMENT_NAME"  # next to api key in console
)

# Index 

**Load data from PDF**

Load PDF using pypdf into array of documents, where each document contains the page content and metadata with page number.

For loading other formats, check the [documentation](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html)

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("PATH_TO_YOUR_FILE")
pages = loader.load_and_split()

**Split the text from pdf into smaller chunks**

There are many ways to split the text. We are using the text splitter that is recommended for generic texts. For more ways to slit the text check the [documentation](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

docs = text_splitter.split_documents(pages)

Create embeddings

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

**Creating a vectorstore**

A vectorstore stores Documents and associated embeddings, and provides fast ways to look up relevant Documents by embeddings. 

There are many ways to create a vectorstore. We are going to use Pinecone. For other types of vectorstores visit the [documentation](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)

First you need to go to [Pinecone](https://www.pinecone.io/) and create an index there. Then type the index name in "index_name"

In [ ]:
from langchain.vectorstores import Pinecone

index_name = "index_name"

#create a new index
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)


If you cannot create a Pinecone account, try to use CromaDB. The following code creates a transient in-memory vectorstore. For further information check the [documentation](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/chroma.html). 

The following code block uses Croma for creating a vectorstore. Uncomment it if you don't have access to pinecone and use it instead.

In [ ]:
# from langchain.vectorstores import Chroma
# docsearch = Chroma.from_documents(docs, embeddings)

Vectorstore is ready. Let's try to query our docsearch with similarity search

In [ ]:
query = "What is DesignOps support model?"
docs = docsearch.similarity_search(query)
# print(len(docs))
# print(docs[0])
print(docs[0].page_content)

# Making a question answering chain
The question answering chain will enable us to generate the answer based on the relevant context chunks. See the [documentation](https://python.langchain.com/en/latest/modules/chains/index_examples/qa_with_sources.html) for more explanation.

In [ ]:
from langchain.chains import RetrievalQA
from langchain import OpenAI

#defining LLM
llm = OpenAI(temperature=0.2)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k": 2}))

Query the chain to test that it's working

In [ ]:
query = "What is DesignOps support model?"
qa.run(query)

# Preparing Zapier tool
First, you need to get a Zapier API key here https://nla.zapier.com/get-started/ and add the the actions that you are going to use in Zapier

In [ ]:
os.environ["ZAPIER_NLA_API_KEY"] = os.environ.get("ZAPIER_NLA_API_KEY", "YOUR_ZAPIER_API_KEY")

Setting up a zapier toolkit. For more information visit [documentation](https://python.langchain.com/en/latest/modules/agents/tools/examples/zapier.html)

In [ ]:
from langchain.agents.agent_toolkits import ZapierToolkit
from langchain.utilities.zapier import ZapierNLAWrapper

zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)

In [ ]:
#If you want to see Zapier toolkit
# tools = toolkit.get_tools()
# print(tools[0])

# Building agent
Assembling it all together into an agent. For more information check the documentation https://python.langchain.com/en/latest/modules/agents/agent_executors/examples/agent_vectorstore.html and https://python.langchain.com/en/latest/modules/agents/tools/examples/zapier.html and https://python.langchain.com/en/latest/modules/agents/agents/examples/conversational_agent.html


In [ ]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI


#defining the tools for the agent
tools = [
    Tool(
        name = "Demo",
        func=qa.run,
        description="use this as the primary source of context information when you are asked the question. Always search for the answers using this tool first, don't make up answers yourself"
    ),
] + toolkit.get_tools()


#setting a memory for conversations
memory = ConversationBufferMemory(memory_key="chat_history")

#Setting up the agent 
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

Quering the agent

*To get an agent do what you want, the prompt should be constructed properly. For user facing apps, we need to look at prompt templates and also figure out if chat is the best interface*

In [ ]:
agent_chain.run(input="What Adrienne Allnutt have said about DesignOps?")

In [ ]:
agent_chain.run(input="Email the answer to email@gmail.com and mention that this email was sent by AI")